In [1]:
%run Improved-model.ipynb

0    23364
1     6636
Name: default payment next month, dtype: int64
(30000, 25)


/home/jui/.pyenv/versions/3.10.12/envs/myenv/lib/python3.10/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(


2    18112
1    18112
Name: SEX, dtype: int64
PR-AUC: 0.5775005059302372
Model Accuracy: 0.8419
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      7305
           1       0.66      0.37      0.47      1751

    accuracy                           0.84      9056
   macro avg       0.76      0.66      0.69      9056
weighted avg       0.82      0.84      0.82      9056


Final Training Log Loss: 0.3026
Final Test Log Loss: 0.3941


In [2]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2025-05-15 13:10:41.958668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747339841.972684 1590750 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747339841.976981 1590750 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-15 13:10:41.991170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from responsibleai.feature_metadata import FeatureMetadata
import seaborn as sns
import matplotlib.pyplot as plt

# Set up feature metadata for RAIInsights
feature_metadata = FeatureMetadata(categorical_features=[], dropped_features=[])

# Add the target feature back to the datasets
X_train_og_with_target = X_train.copy()
X_train_og_with_target[target_feature] = y_train

X_test_og_with_target = X_test.copy()
X_test_og_with_target[target_feature] = y_test

#sample_df = X_test_og_with_target.sample(n=5000, random_state=10)

# Specify the stratification variable
stratify_by = 'AGE'

# Split the data into training and testing sets, with stratification
train, test = train_test_split(X_train_og_with_target, test_size=0.25, stratify=X_train_og_with_target[stratify_by])
sample_df, _ = train_test_split(
    train, 
    train_size=5000/len(train),  # Ensure we get exactly 5000 samples
    stratify=train[stratify_by],  # Maintain distribution
    random_state=42
)

# Check the distribution of the stratification variable in the training and testing sets
print("Train dataset:\n", sample_df[stratify_by].value_counts())

Train dataset:
 2    2246
3    1516
4     648
1     440
5     146
6       4
Name: AGE, dtype: int64


In [4]:
# Now, pass these modified DataFrames to RAIInsights
sample_df[target_feature] = y_test[:5000]
rai_insights = RAIInsights(model_improved, X_train_og_with_target, sample_df, target_feature, 'classification', feature_metadata=feature_metadata)

In [5]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()

# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
                # strings of categorical feature names
#rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')


In [6]:
# Compute: Perform all tasks (this remains CPU-bound)
rai_insights.compute()

Causal Effects
Current Status: Generating Causal Effects.
Current Status: Finished generating causal effects.
Time taken: 0.0 min 0.00010416004806756973 sec
Counterfactual
Time taken: 0.0 min 3.327522426843643e-05 sec
Error Analysis
Current Status: Generating error analysis reports.
Current Status: Finished generating error analysis reports.
Time taken: 0.0 min 0.37671971414238214 sec
Explanations
Current Status: Explaining 22 features
Current Status: Explained 22 features.
Time taken: 0.0 min 0.9462759457528591 sec


In [7]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:8705
